##### Bibliotecas


In [47]:
import os
import numpy as np
import pandas as pd

##### Globais

In [49]:
diretorio        = 'datasets'
arquivo_extensao = '.csv'
lista_arquivos   = []

##### Análise da Estrutura dos Dados (antes da limpeza)

{texto}

In [ ]:
df_dados          = pd.DataFrame()    
df_estrutra_dados = pd.DataFrame()

# Lista os arquivos do diretório
for arquivo in os.listdir(diretorio):
    if arquivo.endswith(arquivo_extensao):
       lista_arquivos.append(arquivo)

# Carrega o DataFrame 
for dataset in lista_arquivos:
    caminho_arquivo   = os.path.join(diretorio, dataset)
    df_dados          = pd.read_csv(caminho_arquivo, sep=';', low_memory=False)
    df_estrutra_dados = pd.concat([df_estrutra_dados, df_dados], ignore_index=True)

print("Dimensões da base")
print("-----------------")
print("")
print(f"Linhas: {len(df_estrutra_dados)}")
print(f"Colunas: {len(df_estrutra_dados.iloc[0])}")
    
print("")
print("") 
print("Estrutura")
print("---------")
print("")
print(df_estrutra_dados.info())

print("")
print("") 
print("Qtde de campos nulos")
print("--------------------")
print("")
print(df_estrutra_dados.isnull().sum())

# Visualização do DataFrame (Head e Tail)
print(display(df_estrutra_dados)) 

##### Carregamento do Conjunto de Dados (após a limpeza)
{texto}

In [ ]:
df_dados        = pd.DataFrame()
df_dados_limpos = pd.DataFrame()

for dataset in lista_arquivos:
    caminho_arquivo = os.path.join(diretorio, dataset)
    df_dados        = pd.read_csv(caminho_arquivo,
                                  sep=';', thousands = '.', decimal = ',', 
                                  dtype = {'Regiao - Sigla': 'category',
                                           'Estado - Sigla': 'category',
                                           'Valor de Compra': np.float64})
    
    df_dados_limpos = pd.concat([df_dados_limpos, df_dados], ignore_index=True)
    
df_dados_limpos.drop(columns=['Nome da Rua',
                              'Numero Rua',
                              'Complemento',
                              'Bairro',
                              'Cep',
                              'Valor de Compra'], inplace=True)    
df_dados_limpos.info()